In [107]:
import os
import pandas as pd
import numpy as np

# Preprocesamiento

## Lectura y filtrado de datos

In [156]:
data_path = r'/data/dibanez/tesis/IT_Platform_Clean_Data_V3/'
!ls {data_path}

Arequipa  Chachapoyas  Juliaca	Lima  Tacna


In [160]:
data_path_output = r'/data/dibanez/tesis/data_preprocesada/1.filtrada/'
!ls {data_path_output}

In [121]:
!ls {data_path}/Arequipa

UNSA


In [161]:
def preprocesar_data(df, departamento, universidad, tecnologia):
    df_pre = df.copy()
    print(df_pre.shape)
    df_pre = df_pre.set_index('Fecha')
    print(f'{len(df_pre[df_pre.index.duplicated(keep="first")])/len(df_pre)*100:.2}% de duplicados')
    df_pre = df_pre[~df_pre.index.duplicated(keep='first')]
    print(df_pre.shape)
    df_pre = df_pre[df_pre.columns[1:]].resample('H').mean()
    df_pre = df_pre.between_time('06:00:00', '17:00:00', include_end=False)
    print(df_pre.shape)
    print()
    df_day_hour.to_csv(f'{data_path_output}{departamento}_{universidad}_{tecnologia}_agg_day_hour.csv')

In [162]:
%%time

columnas=['arch','Fecha', 'Tamb', 'Tmod', 'Irradiancia', 'Vdc', 'Idc', 'Paparente',
       'Pac', 'Pdc']
for departamento in os.listdir(data_path):
    for universidad in os.listdir(f'{data_path}/{departamento}'):
        for tecnologia in os.listdir(f'{data_path}/{departamento}/{universidad}'):
            print(departamento, universidad, tecnologia)
            df = pd.DataFrame(columns=columnas)
            for arch in os.listdir(f'{data_path}/{departamento}/{universidad}/{tecnologia}'):
                if 'Overall' in arch: continue
                df_temp = pd.read_csv(f'{data_path}/{departamento}/{universidad}/{tecnologia}/{arch}', parse_dates=True)
                df_temp['arch'] = arch
                df_temp['Fecha'] = df_temp['Fecha'].astype(np.datetime64)
                df_temp[columnas[2:]] = df_temp[columnas[2:]].astype(np.float64) 
                df_temp =df_temp[columnas]
                df = df.append(df_temp, ignore_index=True)
            preprocesar_data(df, departamento, universidad, tecnologia)
#             print(df.dtypes)
#             print(df[columnas])
#             break

Lima PUCP CIGS
(520224, 10)
0.048% de duplicados
(519973, 9)
(7029, 8)

Lima PUCP HIT
(521817, 10)
0.048% de duplicados
(521566, 9)
(7029, 8)

Lima PUCP PERC
(521798, 10)
0.048% de duplicados
(521547, 9)
(7029, 8)

Chachapoyas UNTRM CIGS
(211339, 10)
0.0% de duplicados
(211339, 9)
(3145, 8)

Chachapoyas UNTRM HIT
(211470, 10)
0.0% de duplicados
(211470, 9)
(3145, 8)

Chachapoyas UNTRM PERC
(211668, 10)
0.0% de duplicados
(211668, 9)
(3145, 8)

Arequipa UNSA CIGS
(107788, 10)
0.0% de duplicados
(107788, 9)
(6875, 8)

Arequipa UNSA HIT
(110548, 10)
0.0% de duplicados
(110548, 9)
(6875, 8)

Arequipa UNSA PERC
(110231, 10)
0.0% de duplicados
(110231, 9)
(6875, 8)

Tacna UNJBG CIGS
(88486, 10)
0.0% de duplicados
(88486, 9)
(6952, 8)

Tacna UNJBG HIT
(93086, 10)
0.0% de duplicados
(93086, 9)
(6960, 8)

Tacna UNJBG PERC
(91948, 10)
0.0% de duplicados
(91948, 9)
(6959, 8)

Juliaca UNAJ CIGS
(135792, 10)
0.0% de duplicados
(135792, 9)
(2098, 8)

Juliaca UNAJ HIT
(135914, 10)
0.0% de duplicados


## Limpieza de datos

In [256]:
data_path_output = r'/data/dibanez/tesis/data_preprocesada/1.filtrada/'
!ls {data_path_output}

Arequipa_UNSA_CIGS_agg_day_hour.csv	 Juliaca_UNAJ_PERC_agg_day_hour.csv
Arequipa_UNSA_HIT_agg_day_hour.csv	 Lima_PUCP_CIGS_agg_day_hour.csv
Arequipa_UNSA_PERC_agg_day_hour.csv	 Lima_PUCP_HIT_agg_day_hour.csv
Chachapoyas_UNTRM_CIGS_agg_day_hour.csv  Lima_PUCP_PERC_agg_day_hour.csv
Chachapoyas_UNTRM_HIT_agg_day_hour.csv	 Tacna_UNJBG_CIGS_agg_day_hour.csv
Chachapoyas_UNTRM_PERC_agg_day_hour.csv  Tacna_UNJBG_HIT_agg_day_hour.csv
Juliaca_UNAJ_CIGS_agg_day_hour.csv	 Tacna_UNJBG_PERC_agg_day_hour.csv
Juliaca_UNAJ_HIT_agg_day_hour.csv


In [286]:
data_path_llenado_output = r'/data/dibanez/tesis/data_preprocesada/2.llenada/'

In [261]:

!cp {data_path_output}Lima_PUCP_HIT_agg_day_hour.csv lima_vacio_hit.csv


In [276]:
def llenar_vacios_mean_mes(df):
    df_suc = df.copy()
    df_suc['Fecha'] = df_suc['Fecha'].astype(np.datetime64)
    df_suc['str_anho_mes_hora']  = df_suc['Fecha'].dt.year.astype(str) + df_suc['Fecha'].dt.month.astype(str) + df_suc['Fecha'].dt.time.astype(str)
    gb_amh = df_suc.groupby('str_anho_mes_hora').mean()
    gb_amh=gb_amh.reset_index()
    df_fill = pd.merge(gb_amh,df_suc[["str_anho_mes_hora","Fecha"]],on="str_anho_mes_hora").set_index('Fecha')
    df_suc = df_suc.set_index('Fecha')
    df_lleno = df_suc.fillna(value=df_fill)
    df_lleno = df_lleno.reset_index()
    df_lleno = df_lleno[df_lleno.columns[:-1]]
    
#     print(df_fill)
    return df_lleno

In [277]:
df_vacios = pd.read_csv(data_path_output+'Lima_PUCP_HIT_agg_day_hour.csv')
df_llenado = llenar_vacios_mean_mes(df_vacios)
df_llenado.to_csv('lleno_pucp_hit.csv', index=False)

In [289]:
for arch in os.listdir(data_path_output):
    df_vacios = pd.read_csv(data_path_output+arch)
    df_llenado = llenar_vacios_mean_mes(df_vacios)
    df_llenado.to_csv('data_path_llenado_output'+'llenado_'+arch, index=False)

In [288]:
data_path_output = r'/data/dibanez/tesis/data_preprocesada/1.filtrada/'
!ls {data_path_llenado_output}

llenado_Arequipa_UNSA_CIGS_agg_day_hour.csv
llenado_Arequipa_UNSA_HIT_agg_day_hour.csv
llenado_Arequipa_UNSA_PERC_agg_day_hour.csv
llenado_Chachapoyas_UNTRM_CIGS_agg_day_hour.csv
llenado_Chachapoyas_UNTRM_HIT_agg_day_hour.csv
llenado_Chachapoyas_UNTRM_PERC_agg_day_hour.csv
llenado_Juliaca_UNAJ_CIGS_agg_day_hour.csv
llenado_Juliaca_UNAJ_HIT_agg_day_hour.csv
llenado_Juliaca_UNAJ_PERC_agg_day_hour.csv
llenado_Lima_PUCP_CIGS_agg_day_hour.csv
llenado_Lima_PUCP_HIT_agg_day_hour.csv
llenado_Lima_PUCP_PERC_agg_day_hour.csv
llenado_Tacna_UNJBG_CIGS_agg_day_hour.csv
llenado_Tacna_UNJBG_HIT_agg_day_hour.csv
llenado_Tacna_UNJBG_PERC_agg_day_hour.csv


In [293]:
# 


### Ejemplo limpieza de dato

In [243]:
df_suc = pd.read_csv(data_path_output+'Lima_PUCP_HIT_agg_day_hour.csv' )
df_suc['Fecha'] = df_suc['Fecha'].astype(np.datetime64)
# df_suc['str_anho_mes']  = df_suc['Fecha'].dt.year.astype(str) + df_suc['Fecha'].dt.month.astype(str)
df_suc['str_anho_mes_hora']  = df_suc['Fecha'].dt.year.astype(str) + df_suc['Fecha'].dt.month.astype(str) + df_suc['Fecha'].dt.time.astype(str)
# df_suc = df_suc.set_index('Fecha')
df_suc

,Fecha,Tamb,Tmod,Irradiancia,Vdc,Idc,Pac,Pdc,str_anho_mes_hora
0,2020-02-16 06:00:00,NaN,NaN,NaN,NaN,NaN,72.000000,NaN,2020206:00:00
1,2020-02-16 07:00:00,NaN,NaN,NaN,NaN,NaN,244.250000,NaN,2020207:00:00
2,2020-02-16 08:00:00,NaN,NaN,NaN,NaN,NaN,708.750000,NaN,2020208:00:00
3,2020-02-16 09:00:00,NaN,NaN,NaN,NaN,NaN,1060.000000,NaN,2020209:00:00
4,2020-02-16 10:00:00,NaN,NaN,NaN,NaN,NaN,1368.250000,NaN,2020210:00:00
...,...,...,...,...,...,...,...,...,...
6870,2021-11-01 12:00:00,NaN,46.273654,NaN,268.256610,5.030712,1305.711864,1348.779661,20211112:00:00
6871,2021-11-01 13:00:00,NaN,43.528431,NaN,273.124407,4.337169,1145.152542,1183.118644,20211113:00:00
6872,2021-11-01 14:00:00,NaN,37.674118,NaN,277.520667,3.355317,888.833333,930.700000,20211114:00:00
6873,2021-11-01 15:00:00,NaN,31.476071,NaN,281.539333,2.152333,573.616667,604.600000,20211115:00:00


In [244]:
gb_amh = df_suc.groupby('str_anho_mes_hora').mean()
gb_amh=gb_amh.reset_index()
gb_amh

,str_anho_mes_hora,Tamb,Tmod,Irradiancia,Vdc,Idc,Pac,Pdc
0,20201006:00:00,NaN,NaN,NaN,NaN,NaN,109.774194,NaN
1,20201007:00:00,NaN,NaN,NaN,NaN,NaN,382.177419,NaN
2,20201008:00:00,NaN,NaN,NaN,NaN,NaN,716.153226,NaN
3,20201009:00:00,NaN,NaN,NaN,NaN,NaN,969.838710,NaN
4,20201010:00:00,NaN,NaN,NaN,NaN,NaN,1164.782258,NaN
...,...,...,...,...,...,...,...,...
237,2021912:00:00,NaN,NaN,NaN,271.565793,4.924158,1289.996756,1334.621333
238,2021913:00:00,NaN,NaN,NaN,273.922498,4.265909,1125.135661,1165.477109
239,2021914:00:00,NaN,NaN,NaN,279.086414,3.204534,854.628646,892.858973
240,2021915:00:00,NaN,NaN,NaN,280.492442,2.070263,549.354695,580.894000


In [245]:
df_fill = pd.merge(gb_amh,df_suc[["str_anho_mes_hora","Fecha"]],on="str_anho_mes_hora").set_index('Fecha')
df_fill

,str_anho_mes_hora,Tamb,Tmod,Irradiancia,Vdc,Idc,Pac,Pdc
Fecha,,,,,,,,
2020-10-01 06:00:00,20201006:00:00,NaN,NaN,NaN,NaN,NaN,109.774194,NaN
2020-10-02 06:00:00,20201006:00:00,NaN,NaN,NaN,NaN,NaN,109.774194,NaN
2020-10-03 06:00:00,20201006:00:00,NaN,NaN,NaN,NaN,NaN,109.774194,NaN
2020-10-04 06:00:00,20201006:00:00,NaN,NaN,NaN,NaN,NaN,109.774194,NaN
2020-10-05 06:00:00,20201006:00:00,NaN,NaN,NaN,NaN,NaN,109.774194,NaN
...,...,...,...,...,...,...,...,...
2021-09-26 16:00:00,2021916:00:00,NaN,NaN,NaN,276.179655,0.826051,212.282514,231.640013
2021-09-27 16:00:00,2021916:00:00,NaN,NaN,NaN,276.179655,0.826051,212.282514,231.640013
2021-09-28 16:00:00,2021916:00:00,NaN,NaN,NaN,276.179655,0.826051,212.282514,231.640013


In [246]:
df_suc = df_suc.set_index('Fecha')
df_suc

,Tamb,Tmod,Irradiancia,Vdc,Idc,Pac,Pdc,str_anho_mes_hora
Fecha,,,,,,,,
2020-02-16 06:00:00,NaN,NaN,NaN,NaN,NaN,72.000000,NaN,2020206:00:00
2020-02-16 07:00:00,NaN,NaN,NaN,NaN,NaN,244.250000,NaN,2020207:00:00
2020-02-16 08:00:00,NaN,NaN,NaN,NaN,NaN,708.750000,NaN,2020208:00:00
2020-02-16 09:00:00,NaN,NaN,NaN,NaN,NaN,1060.000000,NaN,2020209:00:00
2020-02-16 10:00:00,NaN,NaN,NaN,NaN,NaN,1368.250000,NaN,2020210:00:00
...,...,...,...,...,...,...,...,...
2021-11-01 12:00:00,NaN,46.273654,NaN,268.256610,5.030712,1305.711864,1348.779661,20211112:00:00
2021-11-01 13:00:00,NaN,43.528431,NaN,273.124407,4.337169,1145.152542,1183.118644,20211113:00:00
2021-11-01 14:00:00,NaN,37.674118,NaN,277.520667,3.355317,888.833333,930.700000,20211114:00:00


In [252]:
df_lleno = df_suc.fillna(value=df_fill)
df_lleno.isna().sum()/len(df_lleno)

Tamb                 1.000000
Tmod                 0.851200
Irradiancia          1.000000
Vdc                  0.753600
Idc                  0.753600
Pac                  0.008873
Pdc                  0.753600
str_anho_mes_hora    0.000000
dtype: float64

In [249]:
df_suc.isna().sum()/len(df_suc)

Tamb                 1.000000
Tmod                 0.946036
Irradiancia          1.000000
Vdc                  0.788800
Idc                  0.788800
Pac                  0.080582
Pdc                  0.788800
str_anho_mes_hora    0.000000
dtype: float64

In [201]:
loc['20201006:00:00']['Tamb']

nan

In [ ]:
def llenar_vacios(df, ls_cols):
    

## Ejemplo de lectura y filtro

In [115]:
!ls {data_path}/Arequipa

UNSA


In [312]:
dir_data_arequipa = f'3. Datos filtrados v5/'

In [137]:
%%time

anho_ini = 2015
anho_fin = 2015

df = pd.DataFrame(columns=['Fecha', 'nombre_arch', 'Tamb', 'Tmod', 'Irradiancia', 'Vdc', 'Idc', 'Paparente',
       'Pac', 'Pdc', 'Pac*', 'Pdc*', 'TONC'])

for anho in range(anho_ini, anho_fin+1):
  anho = str(anho)
  for mes in os.listdir(data_path + dir_data_arequipa + anho):
    for arch in os.listdir(data_path + dir_data_arequipa + anho + '/'+ mes):
      if 'Resumen' in arch:
        continue
      df_temp = pd.read_excel(data_path + dir_data_arequipa + anho + '/'+ mes + '/' + arch, usecols = "B:M", parse_dates=True) 
      df_temp['nombre_arch'] = arch
        
      df = df.append(df_temp, ignore_index=True)
      print(arch, df_temp.shape)
  

NameError: name 'dir_data_arequipa' is not defined

In [314]:
df.shape

(1086716, 13)

In [315]:
df.head()

,Fecha,nombre_arch,Tamb,Tmod,Irradiancia,Vdc,Idc,Paparente,Pac,Pdc,Pac*,Pdc*,TONC
0,2015-08-02 00:00:12,02_ago_2015.xls,12.3,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2015-08-02 00:00:27,02_ago_2015.xls,12.2,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2015-08-02 00:00:42,02_ago_2015.xls,12.2,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2015-08-02 00:00:57,02_ago_2015.xls,12.2,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2015-08-02 00:01:12,02_ago_2015.xls,12.2,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Preprocesamiento

In [316]:
df_pre = df.copy()
print(df_pre.shape)
df_pre = df_pre.set_index('Fecha')
df_pre.head()

(1086716, 13)


,nombre_arch,Tamb,Tmod,Irradiancia,Vdc,Idc,Paparente,Pac,Pdc,Pac*,Pdc*,TONC
Fecha,,,,,,,,,,,,
2015-08-02 00:00:12,02_ago_2015.xls,12.3,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2015-08-02 00:00:27,02_ago_2015.xls,12.2,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2015-08-02 00:00:42,02_ago_2015.xls,12.2,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2015-08-02 00:00:57,02_ago_2015.xls,12.2,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2015-08-02 00:01:12,02_ago_2015.xls,12.2,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [317]:
# Duplicados de un mismo datetime
print(f'{len(df_pre[df_pre.index.duplicated(keep="first")])/len(df_pre)*100:.2}% de duplicados')
df_pre[df_pre.index.duplicated(keep='first')]

0.67% de duplicados


,nombre_arch,Tamb,Tmod,Irradiancia,Vdc,Idc,Paparente,Pac,Pdc,Pac*,Pdc*,TONC
Fecha,,,,,,,,,,,,
2015-06-05 00:00:00,05_jun_2015 .xls,14.4,0.0,0,0.0,0.00,0.0,0.0,0.000,0.000000,0.000000,0.000000
2015-06-05 00:00:00,05_jun_2015 .xls,14.4,0.0,0,0.0,0.00,0.0,0.0,0.000,0.000000,0.000000,0.000000
2015-06-05 00:00:00,05_jun_2015 .xls,14.5,0.0,0,0.0,0.00,0.0,0.0,0.000,0.000000,0.000000,0.000000
2015-06-05 00:01:00,05_jun_2015 .xls,14.6,0.0,0,0.0,0.00,0.0,0.0,0.000,0.000000,0.000000,0.000000
2015-06-05 00:01:00,05_jun_2015 .xls,14.7,0.0,0,0.0,0.00,0.0,0.0,0.000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
2015-06-26 14:33:00,26_jun_2015.xls,26.3,34.8,474,296.3,3.81,1119.7,1119.9,1128.903,1171.566063,1180.984413,34.345992
2015-06-26 14:34:00,26_jun_2015.xls,26.4,34.9,483,295.3,3.83,1124.1,1116.2,1130.999,1168.245329,1183.734366,34.078675
2015-06-26 14:34:00,26_jun_2015.xls,26.4,34.8,481,295.8,3.82,1119.2,1111.5,1129.956,1162.778533,1182.085992,33.970894


In [318]:
#TODO
df_pre = df_pre[~df_pre.index.duplicated(keep='first')]
print(df_pre.shape)

(1079481, 12)


In [319]:
# pd.to_datetime(df_pre.index)

In [320]:
# df_day = df_pre.between_time('06:00:00', '17:00:00', include_end=False)
df_day = df_pre.copy()
# df_day.shape

In [321]:
df_day_hour = df_day[df_day.columns[1:]].resample('H').mean()
print(df_day_hour.shape)
# df_day_hour['2015-06-02']

(5136, 10)


In [322]:
# df_day_hour = df_day_hour.between_time('04:00', '19:59')
df_day_hour = df_day_hour.between_time('06:00:00', '17:00:00', include_end=False)
print(df_day_hour.shape)
df_day_hour.head()

(2354, 10)


,Tamb,Tmod,Vdc,Idc,Paparente,Pac,Pdc,Pac*,Pdc*,TONC
Fecha,,,,,,,,,,
2015-06-01 06:00:00,11.381250,3.621667,140.022083,0.263125,102.358333,96.193750,101.430112,90.335991,95.242236,-1.946540
2015-06-01 07:00:00,16.312083,19.704583,372.860417,1.958667,732.851667,727.398333,738.693508,716.112174,727.210904,29.759003
2015-06-01 08:00:00,20.875417,34.196667,357.738333,3.832333,1353.288333,1352.006667,1368.406129,1412.752943,1429.894220,42.724692
2015-06-01 09:00:00,23.332083,42.231667,344.000000,5.378083,1824.089583,1821.449167,1848.645100,1976.192022,2005.712458,43.037435
2015-06-01 10:00:00,24.255000,39.797500,353.690833,4.412125,1527.037083,1532.564583,1545.110308,1647.339160,1661.086193,43.067483


In [323]:
df_day_hour.to_csv(data_path+'/'+str(anho_ini)+'_agg_day_hour.csv')

In [324]:
df_day_hour.isna().sum()/len(df_day_hour)

Tamb         0.122345
Tmod         0.122345
Vdc          0.122345
Idc          0.122345
Paparente    0.122345
Pac          0.122345
Pdc          0.122345
Pac*         0.122345
Pdc*         0.122345
TONC         0.122345
dtype: float64

In [ ]:
df = pd.DataFrame(columns=['Fecha', 'nombre_arch', 'Tamb', 'Tmod', 'Irradiancia', 'Vdc', 'Idc', 'Paparente',
       'Pac', 'Pdc', 'Pac*', 'Pdc*', 'TONC'])

In [108]:
data_path = r'/data/dibanez/tesis/IT_Platform_Clean_Data_V3/'